In [37]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql import types
from collections import namedtuple

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

25/03/13 16:19:22 WARN Utils: Your hostname, LAPTOP-F5VFQF4J resolves to a loopback address: 127.0.1.1; using 172.26.218.55 instead (on interface eth0)
25/03/13 16:19:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/13 16:19:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/13 16:19:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/13 16:19:28 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
df_green = spark.read.parquet('../tmp/data/pq/green/*/*')

Usando RDDS vamos a recrear el resultado de este SQL:

```sql
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [5]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [8]:
rows = rdd.take(10)
rows

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 25, 7, 30), PULocationID=35, total_amount=36.14),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 31, 8, 27, 10), PULocationID=218, total_amount=15.0),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 19, 9, 45), PULocationID=74, total_amount=8.76),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 2, 49, 37), PULocationID=243, total_amount=0.0),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 28, 10, 19), PULocationID=141, total_amount=44.14),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 31, 18, 0, 49), PULocationID=82, total_amount=13.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 15, 28, 24), PULocationID=74, total_amount=14.43),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 7, 19, 21, 50), PULocationID=97, total_amount=7.8),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 9, 15, 20, 15), PULocationID=56, total_amount=9.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 

In [10]:
row = rows[0]
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 25, 7, 30), PULocationID=35, total_amount=36.14)

In [17]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [18]:
df_result = rdd \
    .filter(filter_outliers) \
    .take(5)

df_result

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 25, 7, 30), PULocationID=35, total_amount=36.14),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 31, 8, 27, 10), PULocationID=218, total_amount=15.0),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 19, 9, 45), PULocationID=74, total_amount=8.76),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 2, 49, 37), PULocationID=243, total_amount=0.0),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 28, 10, 19), PULocationID=141, total_amount=44.14)]

----------------------------

In [20]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [21]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .take(5)

df_result

[((datetime.datetime(2020, 1, 25, 7, 0), 35), (36.14, 1)),
 ((datetime.datetime(2020, 1, 31, 8, 0), 218), (15.0, 1)),
 ((datetime.datetime(2020, 1, 19, 9, 0), 74), (8.76, 1)),
 ((datetime.datetime(2020, 1, 11, 2, 0), 243), (0.0, 1)),
 ((datetime.datetime(2020, 1, 28, 10, 0), 141), (44.14, 1))]

-----------------------

In [23]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [24]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .take(5)

df_result

[((datetime.datetime(2020, 1, 31, 8, 0), 218), (15.0, 1)),
 ((datetime.datetime(2020, 1, 7, 8, 0), 74), (1497.259999999998, 104)),
 ((datetime.datetime(2020, 1, 31, 17, 0), 223), (228.9, 20)),
 ((datetime.datetime(2020, 1, 9, 23, 0), 62), (82.89999999999999, 4)),
 ((datetime.datetime(2020, 1, 13, 11, 0), 168), (238.51, 5))]

------------------------------------

In [28]:
RevenueRow = namedtuple('RevenueFila', ['hour', 'zone', 'revenue', 'count'])

In [29]:
RevenueRow(hour=datetime(year=2020, month=2, day=3, hour=15), zone=15, revenue=1500, count=375)

RevenueFila(hour=datetime.datetime(2020, 2, 3, 15, 0), zone=15, revenue=1500, count=375)

In [30]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [31]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .take(5)

df_result

[RevenueFila(hour=datetime.datetime(2020, 1, 31, 8, 0), zone=218, revenue=15.0, count=1),
 RevenueFila(hour=datetime.datetime(2020, 1, 7, 8, 0), zone=74, revenue=1497.259999999998, count=104),
 RevenueFila(hour=datetime.datetime(2020, 1, 31, 17, 0), zone=223, revenue=228.9, count=20),
 RevenueFila(hour=datetime.datetime(2020, 1, 9, 23, 0), zone=62, revenue=82.89999999999999, count=4),
 RevenueFila(hour=datetime.datetime(2020, 1, 13, 11, 0), zone=168, revenue=238.51, count=5)]

------------------------------------

In [33]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [34]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema)

In [35]:
df_result.write.parquet('../tmp/data/green-revenue')

25/03/13 17:13:27 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

------------------------------------------

In [39]:
duration_rdd = df_green \
    .select(columns) \
    .rdd

In [40]:
rows = duration_rdd.take(10)
rows

[Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 25, 7, 30), PULocationID=35, DOLocationID=93, trip_distance=9.84),
 Row(VendorID=1, lpep_pickup_datetime=datetime.datetime(2020, 1, 31, 8, 27, 10), PULocationID=218, DOLocationID=130, trip_distance=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 19, 9, 45), PULocationID=74, DOLocationID=75, trip_distance=1.47),
 Row(VendorID=1, lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 2, 49, 37), PULocationID=243, DOLocationID=119, trip_distance=1.2),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 28, 10, 19), PULocationID=141, DOLocationID=91, trip_distance=15.58),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 31, 18, 0, 49), PULocationID=82, DOLocationID=82, trip_distance=1.36),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 15, 28, 24), PULocationID=74, DOLocationID=236, trip_distance=1.99),
 Row(VendorID=2, lpep_pickup_datetime=datetime.

In [44]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

df = pd.DataFrame(rows, columns=columns)
df.head()

,VendorID,lpep_pickup_datetime,PULocationID,DOLocationID,trip_distance
0,NaN,2020-01-25 07:30:00,35,93,9.84
1,1.0,2020-01-31 08:27:10,218,130,0.00
2,2.0,2020-01-19 09:45:00,74,75,1.47
3,1.0,2020-01-11 02:49:37,243,119,1.20
4,NaN,2020-01-28 10:19:00,141,91,15.58


In [45]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [46]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [47]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [50]:
df_predicts\
    .select('trip_distance','predicted_duration') \
    .show()

[Stage 21:>                                                         (0 + 1) / 1]

+-------------+------------------+
|trip_distance|predicted_duration|
+-------------+------------------+
|         9.84|              49.2|
|          0.0|               0.0|
|         1.47|              7.35|
|          1.2|               6.0|
|        15.58|              77.9|
|         1.36| 6.800000000000001|
|         1.99|              9.95|
|         0.96|               4.8|
|         1.22|               6.1|
|          0.8|               4.0|
|         4.64|              23.2|
|         2.39|11.950000000000001|
|        13.88|              69.4|
|         1.47|              7.35|
|         0.55|              2.75|
|         7.84|              39.2|
|         1.36| 6.800000000000001|
|         1.17|              5.85|
|          1.4|               7.0|
|         6.14|              30.7|
+-------------+------------------+
only showing top 20 rows

